In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_API_KEY')
INDOX_OPENAI_API_KEY = os.getenv("INDOX_OPENAI_API_KEY")

In [2]:
from indox import IndoxRetrievalAugmentation
from indox.qa_models import MistralQA
from indox.embeddings import HuggingFaceEmbedding
from indox.data_loader_splitter import UnstructuredLoadAndSplit
from indox.data_loader_splitter import ClusteredSplit

In [3]:
# from indox.qa_models import IndoxApiOpenAiQa
# from indox.embeddings import IndoxOpenAIEmbedding

In [3]:
Indox = IndoxRetrievalAugmentation()

In [4]:
# Using Mistral from HuggingFace
mistral_qa = MistralQA(api_key=HUGGINGFACE_API_KEY,model="mistralai/Mistral-7B-Instruct-v0.2")
embed_hf = HuggingFaceEmbedding(model="multi-qa-mpnet-base-cos-v1")

In [6]:
# Using OpenAi for qa and embedding from Indox
# qa_indox_api_openai = IndoxApiOpenAiQa(api_key=INDOX_OPENAI_API_KEY)
# embed_indox_api_openai = IndoxOpenAIEmbedding(api_key=INDOX_OPENAI_API_KEY,model="text-embedding-ada-002")

In [5]:
file_path = "sample.txt"

# Load and Split with Unstructured library

In [34]:
from indox.splitter import semantic_text_splitter

In [35]:
loader_splitter = UnstructuredLoadAndSplit(file_path=file_path,max_chunk_size=400,splitter=semantic_text_splitter)

In [36]:
docs = loader_splitter.load_and_chunk()

In [38]:
docs

["The wife of a rich man fell sick, and as she felt that her endwas drawing near, she called her only daughter to her bedside andsaid, dear child, be good and pious, and then thegood God will always protect you, and I will look down on youfrom heaven and be near you. Thereupon she closed her eyes anddeparted. Every day the maiden went out to her mother's grave,and wept, and she remained pious and good. When winter camethe snow spread a white sheet over the grave, and by the time thespring sun had drawn it off again, the man had taken another wife.The woman had brought with her into the house two daughters,who were beautiful and fair of face, but vile and black of heart.Now began a bad time for the poor step-child. Is the stupid gooseto sit in the parlor with us, they said. He who wants to eat breadmust earn it. Out with the kitchen-wench. They took her prettyclothes away from her, put an old grey bedgown on her, and gaveher wooden shoes. Just look at the proud princess, how deckedout s

In [21]:
from indox.vector_stores import ChromaVectorStore
db = ChromaVectorStore(embedding=embed_hf,collection_name="sample")

In [22]:
Indox.connect_to_vectorstore(db)

In [23]:
Indox.store_in_vectorstore(docs)

In [24]:
query = "How cinderella reach her happy ending?"

In [25]:
response = Indox.answer_question(query=query,qa_model=mistral_qa,generate_clustered_prompts=False,document_relevancy_filter=False)

In [26]:
response[0]

"Cinderella reaches her happy ending by receiving help from a mysterious bird. The bird grants her magical clothes and allows her to attend a royal ball where she catches the eye of the prince. When the prince realizes he cannot find her after the ball, Cinderella's step-sisters falsely accuse her of running away, causing their father to destroy the pear tree where Cinderella had left her beautiful clothes. However, on the third day of the festival, C"

In [27]:
response[1][0]

['by the hearth in the cinders. And as on that account she always\n\nlooked dusty and dirty, they called her cinderella.\n\nIt happened that the father was once going to the fair, and he\n\nasked his two step-daughters what he should bring back for them.\n\nBeautiful dresses, said one, pearls and jewels, said the second.\n\nAnd you, cinderella, said he, what will you have. Father',
 'cinderella expressed a wish, the bird threw down to her what she\n\nhad wished for.\n\nIt happened, however, that the king gave orders for a festival\n\nwhich was to last three days, and to which all the beautiful young\n\ngirls in the country were invited, in order that his son might choose\n\nhimself a bride. When the two step-sisters heard that they too were',
 'know where she was gone. He waited until her father came, and\n\nsaid to him, the unknown maiden has escaped from me, and I\n\nbelieve she has climbed up the pear-tree. The father thought,\n\ncan it be cinderella. And had an axe brought and cut 

In [28]:
len(response[1][0])

5